In [ ]:
%pip install --user scikit-learn joblib
# training.ipynb

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.ensemble import RandomForestClassifier
import joblib 
import os

# 1. 데이터 로드
file_path = '../assignment4/data/pitcher_data.csv'

if not os.path.exists(file_path):
    print(f"오류: 파일을 찾을 수 없습니다. 경로를 확인하세요: {file_path}")
else:
    df = pd.read_csv(file_path)
    print("데이터 로드 성공!")
    
    # 2. 데이터 전처리 (결측치 제거 및 Feature 선택)
    features = ['balls', 'strikes', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up', 'inning']
    target = 'pitch_type'

    # 타겟 (pitch_type)에 결측치가 있는 행은 제거
    df = df.dropna(subset=[target])

    X = df[features]
    y = df[target]

    # 주자 상황(on_1b 등)의 NaN은 주자가 없었음(0)으로 간주합니다.
    X = X.fillna(0) 
    
    # 추가 단계 A: Label Encoding (구종 이름을 숫자로 변환)
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # 인코더 저장 (나중에 추론(inference) 시 숫자를 다시 이름으로 변환하기 위해 필수)
    joblib.dump(le, 'pitch_type_encoder.pkl')
    print("인코더 저장 완료: pitch_type_encoder.pkl")
    
    # 3. 데이터 분할 (Train : Val : Test = 약 7 : 1 : 2)
    # y_encoded 사용
    X_train_val, X_test, y_train_val_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    
    # Validation 세트 분할
    X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(X_train_val, y_train_val_encoded, test_size=0.125, random_state=42)

    print(f"학습용(Train): {X_train.shape}")
    print(f"검증용(Val): {X_val.shape}")
    print(f"평가용(Test): {X_test.shape}")

    # 4. 모델 학습
    print("모델 학습 시작...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train_encoded) 
    print("학습 완료!")

    # 5. 모델 저장 
    joblib.dump(model, 'pitcher_model.pkl')
    print("모델 저장 완료: pitcher_model.pkl")